In [2]:
import time
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver

In [3]:
def init_browser():
    executable_path = {'executable_path': 'chromedriver'}
    return Browser('chrome', **executable_path)

In [4]:
def scrape():
    browser = init_browser()
    info = {}
    url = "https://mars.nasa.gov/news/"
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    info["headline"] = soup.find("div", class_="content_title").get_text()
    info["date"] = soup.find("div", class_="list_date").get_text()
    info["teaser"] = soup.find("div", class_="article_teaser_body").get_text()

    return info

In [5]:
latestNews = scrape()

In [10]:
latestNews

{'date': 'May 23, 2018',
 'headline': 'InSight Steers Toward Mars',
 'teaser': 'The spacecraft has completed its first trajectory correction maneuver.'}

In [11]:
latestHeadline = latestNews['headline']
latestDate = latestNews['date']
latestTeaser = latestNews['teaser']

print ("{} - {}\n{}".format(latestHeadline, latestDate, latestTeaser))

InSight Steers Toward Mars - May 23, 2018
The spacecraft has completed its first trajectory correction maneuver.


In [25]:
def scrape_img():
    browser = init_browser()
    url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    base_url = "https://www.jpl.nasa.gov"
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    img_path = soup.find("a", class_="button fancybox")['data-fancybox-href']
    full_img_url = base_url + img_path
    return full_img_url

In [26]:
scrape_img()

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA15883_ip.jpg'

In [36]:
# div, class="js-tweet-text-container"
def scrape_weather():
    browser = init_browser()
    url = "https://twitter.com/marswxreport?lang=en"
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    latest_weather = soup.find("div", class_="js-tweet-text-container").get_text().replace('\n','')
    return latest_weather


In [37]:
scrape_weather()

'Sol 2058 (May 21, 2018), Sunny, high 4C/39F, low -71C/-95F, pressure at 7.43 hPa, daylight 05:20-17:20'

In [44]:
def scrape_facts():
    url = "https://space-facts.com/mars/"
    tables = pd.read_html(url)
    mars_info = tables[0].rename(columns={0:"categories", 1:"data"})
    return mars_info

In [45]:
scrape_facts()

,categories,data
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [68]:
# scrape href and gather them
# https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars

def scrape_url_list():
    url_list = []
    browser = init_browser()
    url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
    base_url = "https://astrogeology.usgs.gov"
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    for item in soup.find_all("div", class_="item"):
        for a in item.find_all("a", href=True):
            url_list.append(base_url + a['href'])
    url_list = list(set(url_list))
    return url_list

In [69]:
scrape_url_list()

['https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced']

In [71]:
def scrape_href_list():
    href_list = []
    
    url_list = scrape_url_list()
    browser = init_browser()
    for url in url_list:
        browser.visit(url)
        html = browser.html
        soup = BeautifulSoup(html, "html.parser")
        for div in soup.find_all('div', class_="downloads"):
            href_list.append(div.find('a')['href'])
    return href_list

In [72]:
scrape_href_list()

['http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg']